<a href="https://colab.research.google.com/github/Praise-Atadja/EIICD_chatbox/blob/main/CognitiveQuest_chatbox_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **PROJECT NAME:**

##EARLY INTERVENTION INTELLIGENCE FOR COGNITIVE DEVELOPMENT (EIICD) CHATBOX


---

In [1]:
!pip install datasets

In [2]:
pip install transformers[torch]

In [3]:
!pip install accelerate -U

In [4]:
!pip install transformers

In [1]:
# Import necessary libraries
import os
import numpy as np
import tensorflow as tf
import nltk
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer,  DataCollatorWithPadding, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
import torch
import tensorflow_hub as hub
import pandas as pd
import re
from sklearn.metrics import classification_report
import joblib
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
def stemSentence(sentence):
    porter = PorterStemmer()
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

## THE DATASET

In [3]:
from datasets import load_dataset

# Load the dataset from the CSV file
dataset = load_dataset('csv', data_files='/content/drive/MyDrive/Autism Articles/Autsim_Q&A_datatset.csv')

dataset

DatasetDict({
    train: Dataset({
        features: ['Context', 'Questions', 'Answers', 'Patterns', 'Articles'],
        num_rows: 50
    })
})

In [4]:
# Split the dataset into training and validation sets
dataset = dataset['train'].train_test_split(test_size=0.2, seed=42)
train_dataset = dataset['train']
val_dataset = dataset['test']

In [5]:
# Display the number of examples in the training and validation sets
print(f"Number of training examples: {len(train_dataset)}")
print(f"Number of validation examples: {len(val_dataset)}")

Number of training examples: 40
Number of validation examples: 10


In [6]:
print(train_dataset[:5])
print(val_dataset[:5])

{'Context': ['Parents can support by creating structured routines, using visual aids, and providing consistent, positive reinforcement.', 'ABA therapy uses behavior principles to improve social, communication, and learning skills.', 'Siblings can be supported through education about autism, encouragement of open communication, and ensuring they have their own outlets for support and attention.', 'Genetics play a significant role in autism, as evidenced by higher concordance rates among identical twins and the identification of specific genetic mutations associated with autism spectrum disorder. However, not all cases of autism can be explained by genetic factors alone, suggesting a complex interplay of genetic and environmental influences.', "Autism severity levels (mild, moderate, severe) are based on the amount of support an individual requires across different domains (social communication, repetitive behaviors, sensory issues). These levels guide intervention and support strategies

In [7]:
dataset["train"][0]

{'Context': 'Parents can support by creating structured routines, using visual aids, and providing consistent, positive reinforcement.',
 'Questions': 'How can parents support a child with autism at home?',
 'Answers': "Parents can support their child with autism by establishing routines, using visual supports, promoting communication, providing sensory-friendly environments, and participating in therapies and educational activities tailored to their child's needs.",
 'Patterns': 'How do parents help autistic children at home?',
 'Articles': 'https://www.autismspeaks.org/family-support-tool-kit'}

In [8]:
import json

# Convert Dataset objects to lists of dictionaries
dataset_dict = [example for example in train_dataset]
train_data_dict = [example for example in train_dataset]
val_data_dict = [example for example in val_dataset]

# Define file paths
train_file = 'train_dataset.json'
val_file = 'val_dataset.json'
dataset_file = 'dataset.json'

# Save training dataset to JSON
with open(train_file, 'w') as f:
    json.dump(train_data_dict, f, indent=4)

# Save validation dataset to JSON
with open(val_file, 'w') as f:
    json.dump(val_data_dict, f, indent=4)

# Save dataset to JSON
with open(dataset_file, 'w') as f:
    json.dump(dataset_dict, f, indent=4)

print(f"Training dataset saved to {train_file}")
print(f"Validation dataset saved to {val_file}")

Training dataset saved to train_dataset.json
Validation dataset saved to val_dataset.json


In [9]:
print(val_dataset)

Dataset({
    features: ['Context', 'Questions', 'Answers', 'Patterns', 'Articles'],
    num_rows: 10
})


In [10]:
def preprocess_text(text):
    text = text.lower()  # Convert text to lowercase
    text = re.sub(r'[^a-z\s]', '', text)  # Remove special characters and digits
    text = ' '.join(text.split())  # Remove extra whitespaces
    return text

In [18]:
from torch.utils.data import Dataset

# Load the tokenizer
# Define the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def get_unique_answers(dataset):
    """Extracts unique answers from the dataset."""
    unique_answers = set()
    for sample in dataset:
        answer = sample.get('Answers')
        if answer is not None:
            unique_answers.add(answer)
    return unique_answers

# Get unique answers from your dataset
unique_answers = get_unique_answers(train_dataset)  # Assuming train_dataset has all possible answers

# Create label mapping dynamically
label_to_id = {answer: idx for idx, answer in enumerate(unique_answers)}

def tokenize_dataset(dataset, tokenizer=tokenizer, label_to_id=label_to_id):
    input_ids = []
    attention_masks = []
    labels_list = []
    skipped_samples = 0  # Initialize a counter for skipped samples

    for sample in dataset:
        answer = sample.get('Answers')
        if answer is None:
            # Skip samples with missing 'Answers'
            print(f"Warning: 'Answers' key not found or has None value in sample: {sample}")
            skipped_samples += 1  # Increment the counter
            continue

        label = label_to_id.get(answer)
        if label is None:
            print(f"Warning: Label not found for answer '{answer}' in sample: {sample}")
            skipped_samples += 1  # Increment the counter
            continue

        tokenized_example = tokenizer(
            sample.get("Questions", ""),
            sample.get("Context", ""),  # Assuming you changed 'Patterns' to 'Context'
            padding='max_length',
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )

        input_ids.append(tokenized_example['input_ids'].squeeze())
        attention_masks.append(tokenized_example['attention_mask'].squeeze())
        labels_list.append(label)

    if not input_ids:
        raise RuntimeError("No valid samples found. Check your dataset and label mapping.")

    input_ids = torch.stack(input_ids)
    attention_masks = torch.stack(attention_masks)
    labels_tensor = torch.tensor(labels_list, dtype=torch.long)

    return {'input_ids': input_ids, 'attention_mask': attention_masks, 'labels': labels_tensor, 'skipped_samples': skipped_samples}

# Tokenize the datasets
tokenized_train_datasets = tokenize_dataset(train_dataset)


print(f"Number of input_ids: {tokenized_train_datasets['input_ids'].size(0)}")
print(f"Number of attention_masks: {tokenized_train_datasets['attention_mask'].size(0)}")
print(f"Number of labels: {tokenized_train_datasets['labels'].size(0)}")
print(f"Number of skipped samples: {tokenized_train_datasets['skipped_samples']}")



Number of input_ids: 40
Number of attention_masks: 40
Number of labels: 40
Number of skipped samples: 0


In [19]:
# Save tokenize datasets
save_path = 'tokenized_train_datasets'
torch.save(tokenized_train_datasets, save_path)


In [21]:
# Define MyDataset class
class MyDataset(Dataset):
    def __init__(self, tokenized_dataset):
        self.input_ids = tokenized_dataset['input_ids']
        self.attention_mask = tokenized_dataset['attention_mask']
        self.labels = tokenized_dataset['labels']

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

# Prepare the datasets
# Load your own tokenized dataset (example using pickle)
with open('tokenized_train_datasets', 'rb') as f:  # Use the correct filename
    tokenized_train_dataset = torch.load(f)  # Load with torch.load

final_train_dataset = MyDataset(tokenized_train_dataset)

# After creating the datasets, print their lengths
print("Length of final_train_dataset:", len(final_train_dataset))


Length of final_train_dataset: 40


EVALUATING THE MODEL

In [30]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
import pickle

with open('final_train_dataset', 'wb') as f:
    pickle.dump(final_train_dataset, f)

with open('final_train_dataset', 'rb') as f:
    final_train_dataset = pickle.load(f)

# Define model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)

# Use tokenized_train_dataset instead of tokenized_datasets
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(tokenized_train_dataset['labels']))

# Define Trainer
training_args = TrainingArguments(
    output_dir='./output', # Add the output directory
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=100,
    evaluation_strategy="no",
    disable_tqdm=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=final_train_dataset,
    tokenizer=tokenizer
)

# Train the model
trainer.train()

# Save the model
model.save_pretrained('./saved_model')
tokenizer.save_pretrained('./saved_model')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'train_runtime': 839.3204, 'train_samples_per_second': 0.143, 'train_steps_per_second': 0.036, 'train_loss': 3.7453600565592446, 'epoch': 3.0}


('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.txt',
 './saved_model/added_tokens.json')

In [31]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import numpy as np

# Load the tokenizer and model
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained('./saved_model')  # Assuming './saved_model' is where your trained model is saved

# Function to get answer from model
def get_answer(input_text):
    # Tokenize input text
    inputs = tokenizer(input_text, padding=True, truncation=True, max_length=512, return_tensors='pt')

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Convert logits to probabilities using softmax
    probs = torch.softmax(outputs.logits, dim=-1)

    # Get predicted label (answer)
    predicted_label_id = torch.argmax(probs, dim=-1).item()

    # Reverse label_to_id mapping to get answer string
    id_to_label = {v: k for k, v in label_to_id.items()}
    predicted_answer = id_to_label[predicted_label_id]

    return predicted_answer

# Example usage:
input_text = "What is autism spectrum disorder?"
predicted_answer = get_answer(input_text)
print(f"Predicted answer: {predicted_answer}")


Predicted answer: Autism spectrum disorder (ASD) is a developmental disorder that affects communication, behavior, and social interaction. It includes a wide range of conditions characterized by challenges with social skills, repetitive behaviors, speech, and nonverbal communication.


In [ ]:
# Example usage:
input_text = "What is autism spectrum disorder?"
predicted_answer = get_answer(input_text)
print(f"Predicted answer: {predicted_answer}")